# 60/40 Portfolio Improvement Backtesting
## Testing Alternative Asset Class Additions

**Objective:** Test whether adding Infrastructure, Private Credit, and Real Assets improves the traditional 60/40 portfolio.

**Date:** February 2026

---

### Notebook Structure
1. **Setup & Installation**
2. **Data Download** (from yfinance)
3. **Data Processing & Quality Checks**
4. **Portfolio Construction**
5. **Performance Metrics Calculation**
6. **Visualization**
7. **Statistical Testing**
8. **Results & Conclusions**

---
## 1. Setup & Installation

In [ ]:
# Install required packages (run once)
# Uncomment the lines below if packages are not installed

# !pip install yfinance pandas numpy scipy matplotlib seaborn

In [ ]:
# Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

---
## 2. Data Download

### 2.1 Define Tickers and Metadata

**Data Availability Summary:**

| Asset Class | ETF Proxy | Ticker | Available From | Limitation |
|-------------|-----------|--------|----------------|------------|
| US Equity | SPDR S&P 500 | SPY | 1993 | ✅ Good proxy |
| US Bonds | iShares Core Agg | AGG | 2003 | ✅ Good proxy |
| Infrastructure | iShares Global Infra | IGF | 2007 | ⚠️ Listed only (higher vol) |
| Private Credit | iShares High Yield | HYG | 2007 | ⚠️ Fixed rate, not floating |
| Private Credit | Invesco Senior Loan | BKLN | 2011 | ⚠️ Better proxy (floating) |
| Real Assets | SPDR Natural Resources | GNR | 2010 | ⚠️ Listed equities, not land |
| Timber | iShares Timber | WOOD | 2008 | ⚠️ Timber companies, not land |

**❌ NOT available on yfinance:** NCREIF Farmland/Timberland, Cliffwater Direct Lending, EDHECinfra

In [ ]:
# Define configuration
START_DATE = "2000-01-01"  # Request early, actual data starts later
END_DATE = datetime.today().strftime('%Y-%m-%d')

print(f"Data request period: {START_DATE} to {END_DATE}")

In [ ]:
# Define all tickers to download
TICKERS = {
    # CORE PORTFOLIO ASSETS
    'Equity': {
        'ticker': 'SPY',
        'name': 'SPDR S&P 500 ETF',
        'expense_ratio': 0.0945,
    },
    'Bonds': {
        'ticker': 'AGG',
        'name': 'iShares Core US Aggregate Bond ETF',
        'expense_ratio': 0.03,
    },
    
    # ALTERNATIVE ASSETS
    'Infrastructure': {
        'ticker': 'IGF',
        'name': 'iShares Global Infrastructure ETF',
        'expense_ratio': 0.40,
    },
    'Credit_HY': {
        'ticker': 'HYG',
        'name': 'iShares High Yield Corporate Bond ETF',
        'expense_ratio': 0.48,
    },
    'Credit_Loans': {
        'ticker': 'BKLN',
        'name': 'Invesco Senior Loan ETF',
        'expense_ratio': 0.65,
    },
    'RealAssets': {
        'ticker': 'GNR',
        'name': 'SPDR S&P Global Natural Resources ETF',
        'expense_ratio': 0.40,
    },
    'Timber': {
        'ticker': 'WOOD',
        'name': 'iShares Global Timber & Forestry ETF',
        'expense_ratio': 0.40,
    },
    
    # ADDITIONAL REFERENCE
    'REITs': {
        'ticker': 'VNQ',
        'name': 'Vanguard Real Estate ETF',
        'expense_ratio': 0.12,
    },
    'Commodities': {
        'ticker': 'DBC',
        'name': 'Invesco DB Commodity Index Fund',
        'expense_ratio': 0.85,
    },
    'Gold': {
        'ticker': 'GLD',
        'name': 'SPDR Gold Shares',
        'expense_ratio': 0.40,
    },
    'TBills': {
        'ticker': 'BIL',
        'name': 'SPDR Bloomberg 1-3 Month T-Bill ETF',
        'expense_ratio': 0.1356,
    },
}

print(f"Defined {len(TICKERS)} tickers for download")

### 2.2 Download Data from yfinance

In [ ]:
def download_data(tickers_dict, start_date, end_date):
    """
    Download adjusted close prices for all tickers.
    
    Returns:
        pd.DataFrame: Daily adjusted close prices
    """
    # Extract ticker symbols
    ticker_list = [info['ticker'] for info in tickers_dict.values()]
    ticker_names = list(tickers_dict.keys())
    
    print(f"Downloading {len(ticker_list)} tickers...")
    print(f"Tickers: {ticker_list}")
    print()
    
    # Download all at once (faster)
    data = yf.download(
        ticker_list,
        start=start_date,
        end=end_date,
        auto_adjust=True,  # Adjust for splits and dividends
        progress=True
    )
    
    # Extract Close prices
    if isinstance(data.columns, pd.MultiIndex):
        prices = data['Close']
    else:
        prices = data[['Close']]
        prices.columns = ticker_list
    
    # Rename columns to asset class names
    ticker_to_name = {info['ticker']: name for name, info in tickers_dict.items()}
    prices.columns = [ticker_to_name.get(col, col) for col in prices.columns]
    
    return prices

# Download data
daily_prices = download_data(TICKERS, START_DATE, END_DATE)

print(f"\nDownloaded data shape: {daily_prices.shape}")
print(f"Date range: {daily_prices.index[0].strftime('%Y-%m-%d')} to {daily_prices.index[-1].strftime('%Y-%m-%d')}")

In [ ]:
# Preview the data
daily_prices.tail(10)

### 2.3 Convert to Monthly Returns

In [ ]:
def calculate_monthly_returns(daily_prices):
    """
    Convert daily prices to monthly total returns.
    
    Returns:
        pd.DataFrame: Monthly returns
    """
    # Resample to month-end prices
    monthly_prices = daily_prices.resample('ME').last()
    
    # Calculate percentage returns
    monthly_returns = monthly_prices.pct_change()
    
    # Drop first row (NaN)
    monthly_returns = monthly_returns.dropna(how='all')
    
    return monthly_returns

# Calculate monthly returns
monthly_returns = calculate_monthly_returns(daily_prices)

print(f"Monthly returns shape: {monthly_returns.shape}")
print(f"Date range: {monthly_returns.index[0].strftime('%Y-%m')} to {monthly_returns.index[-1].strftime('%Y-%m')}")

In [ ]:
# Preview monthly returns
monthly_returns.tail(10)

---
## 3. Data Processing & Quality Checks

### 3.1 Data Availability Summary

In [ ]:
def data_quality_summary(returns_df):
    """
    Generate data quality summary statistics.
    """
    summary = pd.DataFrame({
        'First_Date': returns_df.apply(lambda x: x.first_valid_index()),
        'Last_Date': returns_df.apply(lambda x: x.last_valid_index()),
        'Total_Months': returns_df.count(),
        'Missing': returns_df.isna().sum(),
        'Ann_Return_%': (returns_df.mean() * 12 * 100).round(2),
        'Ann_Vol_%': (returns_df.std() * np.sqrt(12) * 100).round(2),
        'Min_%': (returns_df.min() * 100).round(2),
        'Max_%': (returns_df.max() * 100).round(2),
    })
    
    # Format dates
    summary['First_Date'] = summary['First_Date'].dt.strftime('%Y-%m')
    summary['Last_Date'] = summary['Last_Date'].dt.strftime('%Y-%m')
    
    return summary

# Generate summary
quality_summary = data_quality_summary(monthly_returns)
quality_summary

### 3.2 Identify Common Analysis Period

In [ ]:
# Define asset groups for different analyses
ANALYSIS_CONFIGS = {
    'Full_Analysis': ['Equity', 'Bonds', 'Infrastructure', 'Credit_HY', 'RealAssets'],
    'With_Floating_Credit': ['Equity', 'Bonds', 'Infrastructure', 'Credit_Loans', 'RealAssets'],
    'Infra_Only': ['Equity', 'Bonds', 'Infrastructure'],
    'Credit_Only': ['Equity', 'Bonds', 'Credit_HY'],
    'Core_Only': ['Equity', 'Bonds'],
}

def find_common_period(returns_df, columns):
    """
    Find the common date range where all specified columns have data.
    """
    available_cols = [c for c in columns if c in returns_df.columns]
    missing_cols = [c for c in columns if c not in returns_df.columns]
    
    if missing_cols:
        return None, None, 0, missing_cols
    
    subset = returns_df[available_cols].dropna()
    
    if subset.empty:
        return None, None, 0, []
    
    return subset.index[0], subset.index[-1], len(subset), []

# Check each configuration
print("Available Analysis Periods:\n")
print(f"{'Configuration':<25} {'Start':<10} {'End':<10} {'Months':<8} {'Status'}")
print("-" * 70)

for config_name, columns in ANALYSIS_CONFIGS.items():
    start, end, months, missing = find_common_period(monthly_returns, columns)
    if missing:
        print(f"{config_name:<25} {'N/A':<10} {'N/A':<10} {0:<8} Missing: {missing}")
    elif start is not None:
        print(f"{config_name:<25} {start.strftime('%Y-%m'):<10} {end.strftime('%Y-%m'):<10} {months:<8} ✅ Ready")
    else:
        print(f"{config_name:<25} {'N/A':<10} {'N/A':<10} {0:<8} ❌ No data")

### 3.3 Create Analysis Dataset

We'll use the **Full_Analysis** configuration with the common period where all assets have data.

In [ ]:
# Select primary assets for analysis
PRIMARY_ASSETS = ['Equity', 'Bonds', 'Infrastructure', 'Credit_HY', 'RealAssets']

# Filter to common period
analysis_returns = monthly_returns[PRIMARY_ASSETS].dropna()

print(f"Analysis dataset:")
print(f"  Assets: {list(analysis_returns.columns)}")
print(f"  Period: {analysis_returns.index[0].strftime('%Y-%m')} to {analysis_returns.index[-1].strftime('%Y-%m')}")
print(f"  Months: {len(analysis_returns)}")

In [ ]:
# Also get risk-free rate for Sharpe ratio calculations
if 'TBills' in monthly_returns.columns:
    rf_returns = monthly_returns['TBills'].reindex(analysis_returns.index)
    # Fill missing with 0 (conservative assumption)
    rf_returns = rf_returns.fillna(0)
    print(f"Risk-free rate data: {rf_returns.count()} months")
else:
    # Use constant 2% annual rate if no T-bill data
    rf_returns = pd.Series(0.02/12, index=analysis_returns.index)
    print("Using constant 2% annual risk-free rate (T-bill data not available)")

---
## 4. Portfolio Construction

### 4.1 Define Portfolio Weights

In [ ]:
# Portfolio definitions
# Order: [Equity, Bonds, Infrastructure, Credit, RealAssets]

PORTFOLIOS = {
    'A_Baseline_60_40': {
        'weights': [0.60, 0.40, 0.00, 0.00, 0.00],
        'description': 'Traditional 60/40 portfolio',
        'color': 'black',
    },
    'B_Plus_Infra': {
        'weights': [0.60, 0.30, 0.10, 0.00, 0.00],
        'description': '60/30/10 with Infrastructure',
        'color': 'blue',
    },
    'C_Plus_Credit': {
        'weights': [0.60, 0.30, 0.00, 0.10, 0.00],
        'description': '60/30/10 with Private Credit',
        'color': 'green',
    },
    'D_Plus_RealAssets': {
        'weights': [0.60, 0.30, 0.00, 0.00, 0.10],
        'description': '60/30/10 with Real Assets',
        'color': 'orange',
    },
    'E_Infra_Credit': {
        'weights': [0.60, 0.25, 0.08, 0.07, 0.00],
        'description': '60/25/8/7 Infrastructure + Credit',
        'color': 'purple',
    },
    'F_All_Three': {
        'weights': [0.60, 0.22, 0.06, 0.06, 0.06],
        'description': '60/22/6/6/6 All Three Alternatives',
        'color': 'red',
    },
    'G_Aggressive': {
        'weights': [0.55, 0.20, 0.10, 0.08, 0.07],
        'description': '55/20/10/8/7 Aggressive Alternatives',
        'color': 'darkred',
    },
}

# Display portfolio allocations
print("Portfolio Allocations:\n")
print(f"{'Portfolio':<20} {'Equity':>8} {'Bonds':>8} {'Infra':>8} {'Credit':>8} {'Real':>8} {'Total':>8}")
print("-" * 80)

for name, config in PORTFOLIOS.items():
    w = config['weights']
    total = sum(w)
    print(f"{name:<20} {w[0]*100:>7.0f}% {w[1]*100:>7.0f}% {w[2]*100:>7.0f}% {w[3]*100:>7.0f}% {w[4]*100:>7.0f}% {total*100:>7.0f}%")

### 4.2 Calculate Portfolio Returns

In [ ]:
def calculate_portfolio_returns(returns_df, weights):
    """
    Calculate portfolio returns given asset returns and weights.
    Assumes monthly rebalancing to target weights.
    
    Parameters:
        returns_df: DataFrame with asset returns
        weights: list of weights [Equity, Bonds, Infra, Credit, RealAssets]
        
    Returns:
        pd.Series: Portfolio returns
    """
    weights_array = np.array(weights)
    portfolio_returns = (returns_df * weights_array).sum(axis=1)
    return portfolio_returns

# Calculate returns for all portfolios
portfolio_returns = pd.DataFrame()

for name, config in PORTFOLIOS.items():
    portfolio_returns[name] = calculate_portfolio_returns(analysis_returns, config['weights'])

print(f"Calculated returns for {len(PORTFOLIOS)} portfolios")
print(f"Period: {portfolio_returns.index[0].strftime('%Y-%m')} to {portfolio_returns.index[-1].strftime('%Y-%m')}")
print(f"Months: {len(portfolio_returns)}")

In [ ]:
# Preview portfolio returns
portfolio_returns.tail(10)

### 4.3 Calculate Cumulative Wealth

In [ ]:
# Calculate cumulative wealth (starting with $100)
initial_wealth = 100
cumulative_wealth = initial_wealth * (1 + portfolio_returns).cumprod()

print("Final Wealth (starting with $100):\n")
final_wealth = cumulative_wealth.iloc[-1].sort_values(ascending=False)
for name, value in final_wealth.items():
    print(f"  {name:<20}: ${value:,.2f}")

---
## 5. Performance Metrics Calculation

### 5.1 Define Metric Functions

In [ ]:
def annualized_return(returns):
    """Calculate annualized return from monthly returns."""
    total_return = (1 + returns).prod() - 1
    n_years = len(returns) / 12
    return (1 + total_return) ** (1 / n_years) - 1

def annualized_volatility(returns):
    """Calculate annualized volatility from monthly returns."""
    return returns.std() * np.sqrt(12)

def sharpe_ratio(returns, rf_returns):
    """Calculate Sharpe ratio."""
    excess_returns = returns - rf_returns
    if excess_returns.std() == 0:
        return 0
    return (excess_returns.mean() * 12) / (excess_returns.std() * np.sqrt(12))

def sortino_ratio(returns, rf_returns, target=0):
    """Calculate Sortino ratio (penalizes only downside volatility)."""
    excess_returns = returns - rf_returns
    downside_returns = excess_returns[excess_returns < target]
    if len(downside_returns) == 0 or downside_returns.std() == 0:
        return np.nan
    downside_std = downside_returns.std() * np.sqrt(12)
    return (excess_returns.mean() * 12) / downside_std

def max_drawdown(returns):
    """Calculate maximum drawdown."""
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdowns = cumulative / rolling_max - 1
    return drawdowns.min()

def calmar_ratio(returns):
    """Calculate Calmar ratio (return / max drawdown)."""
    ann_ret = annualized_return(returns)
    mdd = abs(max_drawdown(returns))
    if mdd == 0:
        return np.nan
    return ann_ret / mdd

def var_95(returns):
    """Calculate 95% Value at Risk (monthly)."""
    return returns.quantile(0.05)

def cvar_95(returns):
    """Calculate 95% Conditional VaR (Expected Shortfall)."""
    var = var_95(returns)
    return returns[returns <= var].mean()

print("Metric functions defined successfully!")

### 5.2 Calculate All Metrics

In [ ]:
def calculate_all_metrics(portfolio_returns, rf_returns):
    """
    Calculate comprehensive metrics for all portfolios.
    """
    metrics = {}
    
    for col in portfolio_returns.columns:
        returns = portfolio_returns[col]
        rf = rf_returns.reindex(returns.index).fillna(0)
        
        metrics[col] = {
            'Ann_Return_%': annualized_return(returns) * 100,
            'Ann_Volatility_%': annualized_volatility(returns) * 100,
            'Sharpe_Ratio': sharpe_ratio(returns, rf),
            'Sortino_Ratio': sortino_ratio(returns, rf),
            'Max_Drawdown_%': max_drawdown(returns) * 100,
            'Calmar_Ratio': calmar_ratio(returns),
            'VaR_95_%': var_95(returns) * 100,
            'CVaR_95_%': cvar_95(returns) * 100,
            'Total_Return_%': ((1 + returns).prod() - 1) * 100,
            'Positive_Months_%': (returns > 0).mean() * 100,
        }
    
    return pd.DataFrame(metrics).T

# Calculate metrics
metrics_df = calculate_all_metrics(portfolio_returns, rf_returns)

# Round for display
metrics_display = metrics_df.round(2)
metrics_display

### 5.3 Calculate Improvement vs Baseline

In [ ]:
# Calculate improvement vs baseline (Portfolio A)
baseline = metrics_df.loc['A_Baseline_60_40']

improvement_df = metrics_df.copy()
improvement_df['Return_Diff'] = metrics_df['Ann_Return_%'] - baseline['Ann_Return_%']
improvement_df['Sharpe_Diff'] = metrics_df['Sharpe_Ratio'] - baseline['Sharpe_Ratio']
improvement_df['MaxDD_Diff'] = metrics_df['Max_Drawdown_%'] - baseline['Max_Drawdown_%']

print("Improvement vs 60/40 Baseline:\n")
print(improvement_df[['Ann_Return_%', 'Return_Diff', 'Sharpe_Ratio', 'Sharpe_Diff', 'Max_Drawdown_%', 'MaxDD_Diff']].round(2))

### 5.4 Correlation Analysis

In [ ]:
# Calculate correlation matrix for underlying assets
asset_correlation = analysis_returns.corr()

print("Asset Class Correlation Matrix:\n")
print(asset_correlation.round(2))

---
## 6. Visualization

### 6.1 Cumulative Wealth Chart

In [ ]:
# Plot cumulative wealth
fig, ax = plt.subplots(figsize=(14, 8))

for name, config in PORTFOLIOS.items():
    ax.plot(cumulative_wealth.index, cumulative_wealth[name], 
            label=f"{name}: ${cumulative_wealth[name].iloc[-1]:,.0f}",
            color=config['color'],
            linewidth=2 if name == 'A_Baseline_60_40' else 1.5,
            linestyle='-' if name == 'A_Baseline_60_40' else '--')

ax.set_title('Cumulative Wealth: $100 Initial Investment', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Portfolio Value ($)')
ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)

# Add recession shading for 2008-2009 and 2020
ax.axvspan('2008-01-01', '2009-06-30', alpha=0.1, color='red', label='GFC')
ax.axvspan('2020-02-01', '2020-04-30', alpha=0.1, color='red', label='COVID')
ax.axvspan('2022-01-01', '2022-12-31', alpha=0.1, color='orange', label='2022 Inflation')

plt.tight_layout()
plt.show()

### 6.2 Drawdown Chart

In [ ]:
def calculate_drawdown_series(returns):
    """Calculate drawdown series."""
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdowns = cumulative / rolling_max - 1
    return drawdowns

# Calculate drawdowns for all portfolios
drawdowns = pd.DataFrame()
for col in portfolio_returns.columns:
    drawdowns[col] = calculate_drawdown_series(portfolio_returns[col])

# Plot drawdowns
fig, ax = plt.subplots(figsize=(14, 6))

# Plot only key portfolios for clarity
key_portfolios = ['A_Baseline_60_40', 'B_Plus_Infra', 'E_Infra_Credit', 'F_All_Three']

for name in key_portfolios:
    config = PORTFOLIOS[name]
    ax.fill_between(drawdowns.index, drawdowns[name] * 100, 0,
                    alpha=0.3, label=name, color=config['color'])
    ax.plot(drawdowns.index, drawdowns[name] * 100, color=config['color'], linewidth=1)

ax.set_title('Portfolio Drawdowns Over Time', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Drawdown (%)')
ax.legend(loc='lower left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.3 Risk-Return Scatter Plot

In [ ]:
# Risk-Return scatter plot
fig, ax = plt.subplots(figsize=(10, 8))

for name, config in PORTFOLIOS.items():
    x = metrics_df.loc[name, 'Ann_Volatility_%']
    y = metrics_df.loc[name, 'Ann_Return_%']
    ax.scatter(x, y, s=150, color=config['color'], label=name, alpha=0.8)
    ax.annotate(name.split('_')[0], (x, y), textcoords="offset points", 
                xytext=(5, 5), fontsize=9)

ax.set_title('Risk-Return Profile: All Portfolios', fontsize=14, fontweight='bold')
ax.set_xlabel('Annualized Volatility (%)')
ax.set_ylabel('Annualized Return (%)')
ax.legend(loc='lower right', fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.4 Correlation Heatmap

In [ ]:
# Correlation heatmap
fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(asset_correlation, annot=True, cmap='RdYlGn_r', center=0,
            fmt='.2f', square=True, linewidths=0.5, ax=ax,
            vmin=-1, vmax=1)

ax.set_title('Asset Class Correlation Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 6.5 Performance Metrics Bar Chart

In [ ]:
# Bar chart comparing key metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colors = [PORTFOLIOS[name]['color'] for name in metrics_df.index]

# Annualized Return
ax1 = axes[0, 0]
metrics_df['Ann_Return_%'].plot(kind='bar', ax=ax1, color=colors)
ax1.set_title('Annualized Return (%)', fontweight='bold')
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
ax1.axhline(y=metrics_df.loc['A_Baseline_60_40', 'Ann_Return_%'], color='black', linestyle='--', label='Baseline')

# Sharpe Ratio
ax2 = axes[0, 1]
metrics_df['Sharpe_Ratio'].plot(kind='bar', ax=ax2, color=colors)
ax2.set_title('Sharpe Ratio', fontweight='bold')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.axhline(y=metrics_df.loc['A_Baseline_60_40', 'Sharpe_Ratio'], color='black', linestyle='--', label='Baseline')

# Max Drawdown
ax3 = axes[1, 0]
metrics_df['Max_Drawdown_%'].plot(kind='bar', ax=ax3, color=colors)
ax3.set_title('Maximum Drawdown (%)', fontweight='bold')
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=45, ha='right')
ax3.axhline(y=metrics_df.loc['A_Baseline_60_40', 'Max_Drawdown_%'], color='black', linestyle='--', label='Baseline')

# Volatility
ax4 = axes[1, 1]
metrics_df['Ann_Volatility_%'].plot(kind='bar', ax=ax4, color=colors)
ax4.set_title('Annualized Volatility (%)', fontweight='bold')
ax4.set_xticklabels(ax4.get_xticklabels(), rotation=45, ha='right')
ax4.axhline(y=metrics_df.loc['A_Baseline_60_40', 'Ann_Volatility_%'], color='black', linestyle='--', label='Baseline')

plt.tight_layout()
plt.show()

---
## 7. Statistical Testing

### 7.1 Paired T-Test for Return Improvement

In [ ]:
def test_return_improvement(portfolio_returns, baseline_col='A_Baseline_60_40'):
    """
    Perform paired t-test to check if portfolios significantly outperform baseline.
    """
    baseline = portfolio_returns[baseline_col]
    results = []
    
    for col in portfolio_returns.columns:
        if col == baseline_col:
            continue
            
        portfolio = portfolio_returns[col]
        
        # Paired t-test
        t_stat, p_value = stats.ttest_rel(portfolio, baseline)
        
        # Calculate mean difference
        mean_diff = (portfolio - baseline).mean() * 12 * 100  # Annualized %
        
        results.append({
            'Portfolio': col,
            'Mean_Diff_%': mean_diff,
            't_statistic': t_stat,
            'p_value': p_value,
            'Significant_5%': 'Yes' if p_value < 0.05 else 'No',
            'Significant_10%': 'Yes' if p_value < 0.10 else 'No',
        })
    
    return pd.DataFrame(results)

# Run t-tests
ttest_results = test_return_improvement(portfolio_returns)
print("Paired T-Test Results (vs 60/40 Baseline):\n")
print(ttest_results.round(4))

### 7.2 Bootstrap Confidence Intervals

In [ ]:
def bootstrap_sharpe_ratio(returns, rf_returns, n_bootstrap=5000, confidence=0.95):
    """
    Calculate bootstrap confidence interval for Sharpe ratio.
    """
    n = len(returns)
    sharpe_ratios = []
    
    for _ in range(n_bootstrap):
        # Sample with replacement
        idx = np.random.choice(n, size=n, replace=True)
        sample_returns = returns.iloc[idx]
        sample_rf = rf_returns.iloc[idx]
        
        # Calculate Sharpe ratio
        sr = sharpe_ratio(sample_returns, sample_rf)
        sharpe_ratios.append(sr)
    
    sharpe_ratios = np.array(sharpe_ratios)
    
    # Calculate confidence interval
    alpha = 1 - confidence
    lower = np.percentile(sharpe_ratios, alpha/2 * 100)
    upper = np.percentile(sharpe_ratios, (1 - alpha/2) * 100)
    
    return {
        'mean': np.mean(sharpe_ratios),
        'std': np.std(sharpe_ratios),
        'lower_95': lower,
        'upper_95': upper,
    }

# Calculate bootstrap CIs for key portfolios
print("Bootstrap 95% Confidence Intervals for Sharpe Ratio:\n")
print(f"{'Portfolio':<25} {'Sharpe':<10} {'95% CI':<20}")
print("-" * 55)

for col in ['A_Baseline_60_40', 'B_Plus_Infra', 'E_Infra_Credit', 'F_All_Three']:
    returns = portfolio_returns[col]
    rf = rf_returns.reindex(returns.index).fillna(0)
    
    bootstrap_result = bootstrap_sharpe_ratio(returns, rf, n_bootstrap=5000)
    
    print(f"{col:<25} {bootstrap_result['mean']:.3f}      [{bootstrap_result['lower_95']:.3f}, {bootstrap_result['upper_95']:.3f}]")

---
## 8. Sub-Period Analysis

### 8.1 Define Analysis Periods

In [ ]:
# Define sub-periods
PERIODS = {
    'Full_Sample': (portfolio_returns.index[0], portfolio_returns.index[-1]),
    'GFC_Crisis': ('2008-01-01', '2009-12-31'),
    'QE_Era': ('2013-01-01', '2019-12-31'),
    'COVID_2020': ('2020-01-01', '2020-12-31'),
    'Inflation_2021_2023': ('2021-01-01', '2023-12-31'),
    'Year_2022': ('2022-01-01', '2022-12-31'),
}

def filter_to_period(returns_df, start_date, end_date):
    """Filter returns to specific period."""
    mask = (returns_df.index >= start_date) & (returns_df.index <= end_date)
    return returns_df[mask]

# Check data availability for each period
print("Data Availability by Period:\n")
for period_name, (start, end) in PERIODS.items():
    filtered = filter_to_period(portfolio_returns, start, end)
    if len(filtered) > 0:
        print(f"  {period_name:<25}: {len(filtered)} months ({filtered.index[0].strftime('%Y-%m')} to {filtered.index[-1].strftime('%Y-%m')})")
    else:
        print(f"  {period_name:<25}: No data available")

### 8.2 Calculate Metrics by Period

In [ ]:
def calculate_period_metrics(portfolio_returns, rf_returns, periods_dict):
    """
    Calculate key metrics for each period.
    """
    all_results = []
    
    for period_name, (start, end) in periods_dict.items():
        period_returns = filter_to_period(portfolio_returns, start, end)
        period_rf = filter_to_period(rf_returns.to_frame(), start, end).iloc[:, 0]
        
        if len(period_returns) < 6:  # Need at least 6 months
            continue
        
        for col in period_returns.columns:
            returns = period_returns[col]
            rf = period_rf.reindex(returns.index).fillna(0)
            
            # Calculate cumulative return for the period
            cum_return = (1 + returns).prod() - 1
            
            all_results.append({
                'Period': period_name,
                'Portfolio': col,
                'Months': len(returns),
                'Cumulative_%': cum_return * 100,
                'Ann_Return_%': annualized_return(returns) * 100 if len(returns) >= 12 else cum_return * 100,
                'Volatility_%': annualized_volatility(returns) * 100,
                'Sharpe': sharpe_ratio(returns, rf),
                'Max_DD_%': max_drawdown(returns) * 100,
            })
    
    return pd.DataFrame(all_results)

# Calculate
period_metrics = calculate_period_metrics(portfolio_returns, rf_returns, PERIODS)

# Display 2022 specifically (critical stress test)
print("\n" + "="*70)
print("CRITICAL: 2022 STRESS TEST RESULTS")
print("="*70 + "\n")

year_2022 = period_metrics[period_metrics['Period'] == 'Year_2022'].copy()
if len(year_2022) > 0:
    year_2022 = year_2022.set_index('Portfolio')[['Cumulative_%', 'Volatility_%', 'Max_DD_%']]
    year_2022 = year_2022.sort_values('Cumulative_%', ascending=False)
    print(year_2022.round(2))
else:
    print("2022 data not available in dataset")

### 8.3 Period Comparison Heatmap

In [ ]:
# Create pivot table for cumulative returns by period
if len(period_metrics) > 0:
    pivot_returns = period_metrics.pivot(index='Portfolio', columns='Period', values='Cumulative_%')
    
    # Reorder columns
    col_order = [c for c in PERIODS.keys() if c in pivot_returns.columns]
    pivot_returns = pivot_returns[col_order]
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(12, 8))
    
    sns.heatmap(pivot_returns, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                linewidths=0.5, ax=ax)
    
    ax.set_title('Cumulative Returns (%) by Period and Portfolio', fontsize=14, fontweight='bold')
    ax.set_xlabel('Period')
    ax.set_ylabel('Portfolio')
    
    plt.tight_layout()
    plt.show()
else:
    print("Insufficient data for period analysis heatmap")

---
## 9. Results Summary & Conclusions

In [ ]:
# Generate final summary
print("="*70)
print("BACKTESTING RESULTS SUMMARY")
print("="*70)
print(f"\nAnalysis Period: {analysis_returns.index[0].strftime('%Y-%m')} to {analysis_returns.index[-1].strftime('%Y-%m')}")
print(f"Total Months: {len(analysis_returns)}")
print()

# Rank portfolios by Sharpe ratio
ranking = metrics_df[['Ann_Return_%', 'Ann_Volatility_%', 'Sharpe_Ratio', 'Max_Drawdown_%']].copy()
ranking['Rank'] = ranking['Sharpe_Ratio'].rank(ascending=False).astype(int)
ranking = ranking.sort_values('Rank')

print("PORTFOLIO RANKINGS (by Sharpe Ratio):\n")
print(ranking.round(2))

# Improvement summary
baseline_sharpe = metrics_df.loc['A_Baseline_60_40', 'Sharpe_Ratio']
baseline_return = metrics_df.loc['A_Baseline_60_40', 'Ann_Return_%']
baseline_dd = metrics_df.loc['A_Baseline_60_40', 'Max_Drawdown_%']

print("\n" + "-"*70)
print("IMPROVEMENT VS 60/40 BASELINE:")
print("-"*70)
print(f"\nBaseline 60/40: Return={baseline_return:.2f}%, Sharpe={baseline_sharpe:.2f}, MaxDD={baseline_dd:.2f}%\n")

for col in ['B_Plus_Infra', 'E_Infra_Credit', 'F_All_Three', 'G_Aggressive']:
    ret_diff = metrics_df.loc[col, 'Ann_Return_%'] - baseline_return
    sharpe_diff = metrics_df.loc[col, 'Sharpe_Ratio'] - baseline_sharpe
    dd_diff = metrics_df.loc[col, 'Max_Drawdown_%'] - baseline_dd
    
    print(f"{col}:")
    print(f"  Return: {'+' if ret_diff > 0 else ''}{ret_diff:.2f}%")
    print(f"  Sharpe: {'+' if sharpe_diff > 0 else ''}{sharpe_diff:.3f}")
    print(f"  MaxDD:  {'+' if dd_diff > 0 else ''}{dd_diff:.2f}% {'(worse)' if dd_diff < 0 else '(better)'}")
    print()

### 9.2 Key Findings

In [ ]:
# Determine best portfolio
best_sharpe_portfolio = metrics_df['Sharpe_Ratio'].idxmax()
best_return_portfolio = metrics_df['Ann_Return_%'].idxmax()
best_dd_portfolio = metrics_df['Max_Drawdown_%'].idxmax()  # Least negative

print("\n" + "="*70)
print("KEY FINDINGS")
print("="*70)

print(f"""
1. BEST RISK-ADJUSTED RETURNS (Sharpe Ratio):
   → {best_sharpe_portfolio}
   → Sharpe: {metrics_df.loc[best_sharpe_portfolio, 'Sharpe_Ratio']:.3f}

2. HIGHEST ABSOLUTE RETURNS:
   → {best_return_portfolio}
   → Return: {metrics_df.loc[best_return_portfolio, 'Ann_Return_%']:.2f}%

3. LOWEST DRAWDOWN:
   → {best_dd_portfolio}
   → Max DD: {metrics_df.loc[best_dd_portfolio, 'Max_Drawdown_%']:.2f}%

4. INFRASTRUCTURE IMPACT (Portfolio B vs A):
   → Return improvement: {metrics_df.loc['B_Plus_Infra', 'Ann_Return_%'] - baseline_return:.2f}%
   → Sharpe improvement: {metrics_df.loc['B_Plus_Infra', 'Sharpe_Ratio'] - baseline_sharpe:.3f}

5. DIVERSIFICATION BENEFIT:
   → Infrastructure correlation to Equity: {asset_correlation.loc['Infrastructure', 'Equity']:.2f}
   → Infrastructure correlation to Bonds: {asset_correlation.loc['Infrastructure', 'Bonds']:.2f}
""")

### 9.3 Limitations & Caveats

In [ ]:
print("""
================================================================================
IMPORTANT LIMITATIONS & CAVEATS
================================================================================

1. PROXY LIMITATIONS:
   • Infrastructure (IGF): Listed infrastructure ETF, NOT private infrastructure
     - Higher volatility than true private infrastructure
     - Higher correlation to equity markets
   
   • Private Credit (HYG): High-yield bonds, NOT direct lending
     - Fixed rate (not floating like true private credit)
     - Different risk/return profile
   
   • Real Assets (GNR): Natural resource EQUITIES, NOT actual farmland/timberland
     - Much higher volatility than NCREIF indices
     - Higher correlation to equity markets

2. DATA LIMITATIONS:
   • ETF history limited (most start 2007-2011)
   • May not capture full GFC drawdown
   • Results may differ with institutional-quality indices

3. IMPLEMENTATION REALITY:
   • True private assets have:
     - 10-15 year lockups (vs. daily liquidity for ETFs)
     - Higher fees (2-4% vs. 0.3-0.5% for ETFs)
     - J-curve effects in early years
   • Backtest assumes instant rebalancing (not realistic for illiquid assets)

4. STATISTICAL CAVEATS:
   • Limited sample size for some periods
   • Past performance does not guarantee future results
   • Regime changes may alter correlations and returns

RECOMMENDATION:
   • Use these results as DIRECTIONAL GUIDANCE
   • Validate with institutional-quality data if available (NCREIF, EDHECinfra)
   • Apply conservative adjustments to ETF proxy results
""")

---
## 10. Export Results

In [ ]:
# Create output directory
import os
os.makedirs('./output', exist_ok=True)

# Export data
monthly_returns.to_csv('./output/monthly_returns_all_assets.csv')
portfolio_returns.to_csv('./output/portfolio_returns.csv')
metrics_df.to_csv('./output/performance_metrics.csv')
asset_correlation.to_csv('./output/correlation_matrix.csv')
quality_summary.to_csv('./output/data_quality_summary.csv')

if len(period_metrics) > 0:
    period_metrics.to_csv('./output/period_metrics.csv', index=False)

print("Results exported to ./output/ directory:")
print("  • monthly_returns_all_assets.csv")
print("  • portfolio_returns.csv")
print("  • performance_metrics.csv")
print("  • correlation_matrix.csv")
print("  • data_quality_summary.csv")
print("  • period_metrics.csv")

---
## Next Steps

1. **Validate with Better Data:**
   - Obtain NCREIF Timberland/Farmland indices
   - Obtain Cliffwater Direct Lending Index
   - Obtain EDHECinfra for unlisted infrastructure

2. **Sensitivity Analysis:**
   - Test different weight combinations
   - Apply volatility adjustments to proxies
   - Test different rebalancing frequencies

3. **Present Findings:**
   - Prepare board presentation
   - Document methodology and limitations
   - Recommend implementation approach